In [18]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
import random
from sklearn import metrics
from astropy.table import Table

In [2]:
data = np.loadtxt("data_banknote_authentication.txt", delimiter=",")
np.random.shuffle(data)
y = data[:,4]
x = data[:,[0,1,2,3]]
scaler = StandardScaler()
scaler.fit(x)
x = scaler.transform(x)
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.2)
x_train, x_cv, y_train, y_cv = train_test_split(x_train,y_train, test_size=0.25)

In [5]:
logistic_regress = LogisticRegression()
logistic_regress.fit(x_train, y_train)
print("prediction score trainingset: " + str(logistic_regress.score(x_train,y_train,sample_weight=None)))
print("prediction score cross-validationset: " + str(logistic_regress.score(x_cv,y_cv,sample_weight=None)))

prediction score trainingset: 0.979318734793
prediction score cross-validationset: 0.985454545455


In [6]:
param_grid = {'C': np.linspace(0.01, 40, 400), 'max_iter': np.linspace(10, 1000, 100)}
grid_search_lr = GridSearchCV(logistic_regress, param_grid, cv=10)
grid_search_lr.fit(x_train, y_train)
best_C = grid_search_lr.best_params_['C']
best_iter = grid_search_lr.best_params_['max_iter']

print(grid_search_lr.best_score_)
print(grid_search_lr.best_estimator_)

0.987834549878
LogisticRegression(C=11.836616541353385, class_weight=None, dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=10.0,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)


In [9]:
print("predicition score testset: " + str(logistic_regress.score(x_test,y_test,sample_weight=None)))
y_predicted = logistic_regress.predict(x_test)

predicition score testset: 0.981818181818


In [10]:
def make_report(clf_name, clf, x_test, y_test):
    predictions = clf.predict(x_test)
    print("Classification report for " + clf_name + ":\n", classification_report(y_test, predictions))

In [11]:
lr_predictions = make_report("Logistic Regression", log_optimized, x_test, y_test)
lr_preds = log_optimized.fit(x_train, y_train).predict(x_test)
lr_cm = confusion_matrix(y_test, lr_preds)
accuracy_score(y_test, lr_preds)
print("Confusion matrix for Logistic Regression:\n", lr_cm, "\n")


NameError: name 'log_optimized' is not defined

In [17]:
LRconmatrix = metrics.confusion_matrix(y_test,y_predicted)
tn = LRconmatrix[0][0]
tp = LRconmatrix[1][1]
fn = LRconmatrix[1][0]
fp = LRconmatrix[0][1]
pred = tp + fn
p = tp + fn
n = tn + fp
N = tn+tp+fn+fp
print(LRconmatrix)

[[159   4]
 [  1 111]]


In [19]:
table = Table([["error","accuracy","tp-rate","fp-rate","precision"],
           [(fp+fn)/N,(tp+tn)/N,tp/p,fp/n,tp/pred]],names=("evaluator","value"),dtype=None)
table

evaluator,value
str9,float64
error,0.0181818181818
accuracy,0.981818181818
tp-rate,0.991071428571
fp-rate,0.0245398773006
precision,0.991071428571
